<h1><center>Введение в анализ данных</center></h1>
<hr>
<h2><center>Quora Question Pairs</center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

<h2>Подготовка данных:</h2>

<h3>Основная таблица</h3>

In [9]:
filepath = '/media/ivan/Acer/Ivan/sphere/Quora/train.csv'
df = pd.read_csv(filepath, sep=',')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [10]:
df = df.set_index('id')
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [11]:
df.is_duplicate.mean()

0.369197853026293

<h2>Формат выходных данных</h2>

In [3]:
filepath = '/media/ivan/Acer/Ivan/sphere/Quora/sample_submission.csv'
df_out = pd.read_csv(filepath, sep=',')
print len(df_out)
df_out.head()

2345796


,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
